In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',30)
pd.set_option('display.precision',3)

In [25]:
def calc_PA(x):
    return x.AB+x.BB+x.HBP+x.SF+x.SH

def calc_TB(x):
    return x.H+x['2B']+(x['3B'])*2+(x.HR)*3

def calc_AVG(x):
    if x.AB==0:
        return 0
    else:
        return x.H/x.AB

def calc_OBP(x):
    if (x.AB+x.BB+x.HBP+x.SF)==0:
        return 0
    else:
        return (x.H+x.BB+x.HBP)/(x.AB+x.BB+x.HBP+x.SF)

def calc_SLG(x):
    if x.AB==0:
        return 0
    else:
        return x.TB/x.AB  

def calc_OPS(x):
    return x.OBP+x.SLG

In [29]:
df_bat=pd.read_csv('../data/raw/Batting.csv')
df_bat=df_bat[df_bat['yearID']>2000]
df_bat['PA']=df_bat.apply(calc_PA,axis=1)
df_bat['TB']=df_bat.apply(calc_TB,axis=1)
df_bat['AVG']=df_bat.apply(calc_AVG,axis=1)
df_bat['OBP']=df_bat.apply(calc_OBP,axis=1)
df_bat['SLG']=df_bat.apply(calc_SLG,axis=1)
df_bat['OPS']=df_bat.apply(calc_OPS,axis=1)
df_bat.tail()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,PA,TB,AVG,OBP,SLG,OPS
110490,zimmejo02,2021,1,MIL,NL,2,1,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.000,0.000,0.000,0.000
110491,zimmeky01,2021,1,KCA,AL,52,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000,0.000,0.000,0.000
110492,zimmery01,2021,1,WAS,NL,110,255,27,62,16,0,14,46.0,0.0,0.0,16,77.0,0.0,0.0,0.0,2.0,9.0,273.0,120,0.243,0.286,0.471,0.756
110493,zuberty01,2021,1,KCA,AL,31,1,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.000,0.000,0.000,0.000
110494,zuninmi01,2021,1,TBA,AL,109,333,64,72,11,2,33,62.0,0.0,0.0,34,132.0,0.0,7.0,0.0,1.0,7.0,375.0,186,0.216,0.301,0.559,0.860
